In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [4]:
# Model definition
class ConvBlock(nn.Module):
    def __init__(self, in_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=16, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(16)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding='same')
        self.bn2 = nn.BatchNorm1d(16)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=64, kernel_size=1)
        self.bn3 = nn.BatchNorm1d(64)
        self.relu3 = nn.ReLU()
        self.convr = nn.Conv1d(in_channels=in_channels, out_channels=64, kernel_size=1)
    def forward(self, input):
        residual = self.convr(input)
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x += residual
        x = self.relu3(x)
        return x

class IdentityBlock(nn.Module):
    def __init__(self, in_channels):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=16, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(16)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding='same')
        self.bn2 = nn.BatchNorm1d(16)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=64, kernel_size=1)
        self.bn3 = nn.BatchNorm1d(64)
        self.relu3 = nn.ReLU()
    def forward(self, input):
        residual = input
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x += residual
        x = self.relu3(x)
        return x

class ResNet24(nn.Module):
    def __init__(self):
        super(ResNet24, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=6, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3)
        self.pool1 = nn.MaxPool1d(2)
        
        self.convblk1 = ConvBlock(64)
        self.convblk2 = ConvBlock(64)
        self.identityblk1 = IdentityBlock(64)
        self.convblk3 = ConvBlock(64)
        self.identityblk2 = IdentityBlock(64)
        self.convblk4 = ConvBlock(64)
        self.identityblk3 = IdentityBlock(64)
        
        self.pool2 = nn.AvgPool1d(2)
        
        self.fc = nn.Sequential(nn.Flatten(),
                                nn.Linear(in_features=704, out_features=2),
                                nn.Softmax())
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.convblk1(x)
        x = self.convblk2(x)
        x = self.identityblk1(x)
        x = self.convblk3(x)
        x = self.identityblk2(x)
        x = self.convblk4(x)
        x = self.identityblk3(x)
        x = self.pool2(x)
        x = self.fc(x)
        return x


In [5]:
# Create an instance of the model
model = ResNet24().to(device)

print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: ResNet24(
  (conv1): Conv1d(6, 64, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (convblk1): ConvBlock(
    (conv1): Conv1d(64, 16, kernel_size=(1,), stride=(1,))
    (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (conv2): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
    (conv3): Conv1d(16, 64, kernel_size=(1,), stride=(1,))
    (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU()
    (convr): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  )
  (convblk2): ConvBlock(
    (conv1): Conv1d(64, 16, kernel_size=(1,), stride=(1,))
    (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running

In [ ]:
# Hyperparameters
learning_rate = 5e-4
batch_size = 64
epochs = 200
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [6]:
# Implementation
current_acc = 0
last_acc = 0
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluate the model with torch.no_grad()
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    last_acc = current_acc
    current_acc = correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

In [ ]:
triggertimes = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    # Early Stopping
    if current_acc <= last_acc:
        triggertimes += 1
        if triggertimes >= patience:
            print("Early Stopping!")
            break
    else:
        triggertimes = 0
print("Done!")
